In [1]:
import os
import pickle

In [2]:
!pip install -qq transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from bs4 import BeautifulSoup
import torch.nn.functional as F

In [4]:
from dataloader_classes_func import ReviewDataset, createDataLoader

In [5]:
with open("./data/train_reviews_v2.pkl",'rb') as fp:
    train_reviews= pickle.load(fp)

with open("./data/val_reviews_v2.pkl",'rb') as fp:
    val_reviews= pickle.load(fp)
    
with open("./data/train_labels_v2.pkl",'rb') as fp:
    train_labels= pickle.load(fp)

with open("./data/val_labels_v2.pkl",'rb') as fp:
    val_labels= pickle.load(fp)

In [6]:
MODEL_NAME = 'bert-base-cased'
tokenizer= BertTokenizer.from_pretrained(MODEL_NAME)

In [7]:
train_dataset= {'data': train_reviews, 'labels': train_labels}
train_dataloader= createDataLoader(train_dataset, tokenizer)

In [ ]:
val_dataset= {'data': val_reviews, 'labels': val_labels}
val_dataloader= createDataLoader(val_dataset, tokenizer)

In [8]:
class SentimentClassifier(torch.nn.Module):
    
    def __init__(self, n_class=2):
        super(SentimentClassifier, self).__init__()
        self.n_class= n_class
        self.bert_model= BertModel.from_pretrained('bert-base-cased')
        self.drop = torch.nn.Dropout(p=0.3)
        self.out= torch.nn.Linear(self.bert_model.config.hidden_size, self.n_class)
        
    def forward(self, input_ids, attention_mask):
        bert_output= self.bert_model(input_ids= input_ids, attention_mask= attention_mask)
        dropout_output= self.drop(bert_output.pooler_output)
        linear_output= self.out(dropout_output)
        output= F.softmax(linear_output, dim= 1)
        return output

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [10]:
model= SentimentClassifier()
model= model.to(device)

In [11]:
dl_item= next(iter(train_dataloader))
ip_id_tensor= dl_item['encoding']['input_ids'].squeeze().to(device)
attention_mask_tensor= dl_item['encoding']['attention_mask'].squeeze().to(device)

In [13]:
r= model(ip_id_tensor, attention_mask_tensor)

In [14]:
r

tensor([[0.6395, 0.3605],
        [0.5157, 0.4843],
        [0.5654, 0.4346],
        [0.6530, 0.3470],
        [0.4696, 0.5304],
        [0.5468, 0.4532],
        [0.5361, 0.4639],
        [0.5919, 0.4081],
        [0.5566, 0.4434],
        [0.3495, 0.6505],
        [0.6079, 0.3921],
        [0.4881, 0.5119],
        [0.6323, 0.3677],
        [0.5804, 0.4196],
        [0.3897, 0.6103],
        [0.5691, 0.4309],
        [0.6494, 0.3506],
        [0.3820, 0.6180],
        [0.4628, 0.5372],
        [0.7301, 0.2699],
        [0.3659, 0.6341],
        [0.5691, 0.4309],
        [0.3839, 0.6161],
        [0.5655, 0.4345],
        [0.6124, 0.3876],
        [0.4313, 0.5687],
        [0.5690, 0.4310],
        [0.6024, 0.3976],
        [0.5617, 0.4383],
        [0.7061, 0.2939],
        [0.6524, 0.3476],
        [0.6021, 0.3979]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [21]:
r.shape

torch.Size([32, 2])

In [ ]:
def forward_nd_back_prop(model, input_data, criterion, optimizer ):
    ip_id_tensor= input_data['encoding']['input_ids'].squeeze().to(device)
    attention_mask_tensor= input_data['encoding']['attention_mask'].squeeze().to(device)
    labels= input_data['label'].to(device)
    
    model.train()
    optimizer.zero_grad()
    fwd_out= model(ip_id_tensor, attention_mask_tensor)
    model_output= torch.argmax(fwd_out, dim= 1)
    
    correct_preds= model_output.eq(labels).sum().item()
    acc= correct_preds/ len(labels)
    loss= criterion(fwd_out, labels)
    
    loss.backward()
    optimizer.step()
    
    return loss.item(), acc
    
    

In [ ]:
def val_loss_and_acc(model, input_data, criterion):
    ip_id_tensor= input_data['encoding']['input_ids'].squeeze().to(device)
    attention_mask_tensor= input_data['encoding']['attention_mask'].squeeze().to(device)
    labels= input_data['label'].to(device)
    
    model.eval()
    #optimizer.zero_grad()
    fwd_out= model(ip_id_tensor, attention_mask_tensor)
    model_output= torch.argmax(fwd_out, dim= 1)
    
    correct_preds= model_output.eq(labels).sum().item()
    acc= correct_preds/ len(labels)
    loss= criterion(fwd_out, labels)
    
    #loss.backward()
    #optimizer.step()
    
    return loss.item(), acc

In [ ]:
import datetime

In [ ]:
def train(model, 
          train_dataloader, 
          val_dataloader, 
          n_epoch= 10,
          criterion,
          optimizer,
          save_dir="./Saved_models"):
    
    epoch_train_loss=[]
    epoch_val_loss= []
    
    os.makedirs(save_dir, exist_ok= True)
    
    for epoch in range(1, n_epoch+1):
        train_losses= []
        train_acc= []
        val_losses= []
        val_acc= []
        
        
        for batch_i, batch_data in enumerate(train_dataloader, 1):
            train_batch_loss, train_batch_accuracy= forward_nd_back_prop(model, 
                                                                         batch_data, criterion, optimizer )
            val_batch_loss, val_batch_accuracy= val_loss_and_acc(model, batch_data, criterion)
            
            train_losses.append(train_batch_loss)
            train_acc.append(train_batch_accuracy)
            val_losses.append(val_batch_loss)
            val_acc.append(val_batch_accuracy)
            
        print("Epoch ({} / {}) Train_loss: {} , Train_accuracy: {} , Val_loss: {}, Val_acc: {}".
              format(epoch, n_epoch, np.mean(train_losses), np.mean(train_acc), 
                     np.mean(val_losses), np.mean(val_acc)))
        
        curr_train_loss= np.mean(train_losses)
        curr_val_loss= np.mean(val_losses)
        curr_train_acc= np.mean(train_acc)
        curr_val_acc= np.mean(val_acc)
        
        try:
            if min(epoch_val_loss) > curr_val_loss:
                epoch_val_loss.append(curr_val_loss)
                print("Saving the model")
                model_name= "M_"+ str(datetime.datetime.now()).replace(" ", "_")+\
                            "train_{:.3f}_val{:.3f}.pt".format(curr_train_acc, curr_val_acc)
                torch.save(model.state_dict(), model_name)
        except:
            pass
        
        epoch_val_loss.append(curr_val_loss)
        epoch_train_loss.append(curr_train_loss)
        

In [ ]:
optimizer= AdamW(model.parameters(), lr=2e-5, correct_bias=False)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
train(model, train_dataloader, val_dataloader, n_epoch= 3, loss_fn, optimizer)

In [11]:
!nvidia-smi

Wed Feb  3 10:36:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   28C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------